In [8]:
q = ""
for a in range(0, 80):
    q += 'surv.a{:02d} AS "c2_A{:02d}",\n'.format(a, a)
    
q += 'surv.A80+ AS "c2_A80+",\n'
q += 'surv.unbekannt AS "c2_AUnbekannt",\n'
print(q)

surv.a00 AS "c2_A00",
surv.a01 AS "c2_A01",
surv.a02 AS "c2_A02",
surv.a03 AS "c2_A03",
surv.a04 AS "c2_A04",
surv.a05 AS "c2_A05",
surv.a06 AS "c2_A06",
surv.a07 AS "c2_A07",
surv.a08 AS "c2_A08",
surv.a09 AS "c2_A09",
surv.a10 AS "c2_A10",
surv.a11 AS "c2_A11",
surv.a12 AS "c2_A12",
surv.a13 AS "c2_A13",
surv.a14 AS "c2_A14",
surv.a15 AS "c2_A15",
surv.a16 AS "c2_A16",
surv.a17 AS "c2_A17",
surv.a18 AS "c2_A18",
surv.a19 AS "c2_A19",
surv.a20 AS "c2_A20",
surv.a21 AS "c2_A21",
surv.a22 AS "c2_A22",
surv.a23 AS "c2_A23",
surv.a24 AS "c2_A24",
surv.a25 AS "c2_A25",
surv.a26 AS "c2_A26",
surv.a27 AS "c2_A27",
surv.a28 AS "c2_A28",
surv.a29 AS "c2_A29",
surv.a30 AS "c2_A30",
surv.a31 AS "c2_A31",
surv.a32 AS "c2_A32",
surv.a33 AS "c2_A33",
surv.a34 AS "c2_A34",
surv.a35 AS "c2_A35",
surv.a36 AS "c2_A36",
surv.a37 AS "c2_A37",
surv.a38 AS "c2_A38",
surv.a39 AS "c2_A39",
surv.a40 AS "c2_A40",
surv.a41 AS "c2_A41",
surv.a42 AS "c2_A42",
surv.a43 AS "c2_A43",
surv.a44 AS "c2_A44",
surv.a45 A

In [2]:
import loadenv
import os
import db
import pandas as pd
import pandas.io.sql as sqlio
from db import get_connection

In [3]:
conn, cur = get_connection()

dat = sqlio.read_sql_query("""
SELECT ags, age, SUM(population)
FROM population_zensus
GROUP BY ags, age
ORDER BY ags, age
""", conn)


for _, row in dat.iterrows():
    d = row.to_dict()
    
    if d['age'] == 0:
        cur.execute(f"INSERT INTO population_survstat_agegroup (ags, a00) VALUES ('{'{:05d}'.format(int(d['ags']))}', {d['sum']})")
    elif d['age'] > 0 and d['age'] < 80:
        cur.execute(f"UPDATE population_survstat_agegroup SET {'a{:02d}'.format(int(d['age']))} = {d['sum']} WHERE ags = '{'{:05d}'.format(int(d['ags']))}'")
        
conn.commit()

In [13]:
conn, cur = get_connection()
for ags, row in dat[dat.age >= 80].groupby(['ags']).sum().iterrows():
    d = row.to_dict()
    cur.execute(f"UPDATE population_survstat_agegroup SET \"A80+\" = {d['sum']} WHERE ags = '{'{:05d}'.format(int(ags))}'")
        
conn.commit()